In [21]:
import psycopg2
import psycopg2.extras
import os
#import pygwalker as pyg

def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()
    
    
def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

In [22]:
query_filtro_data="SELECT *, CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) AS sortable_date_int FROM expresso_2024 WHERE centro = '8819009' AND CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) > 20240101;"
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = query_filtro_data
#query = "select * from expresso_2023 where centro='8819009'"
cur.execute(query)
results_expresso_ = cur.fetchall()


In [23]:
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = "select Cp7,longitude_x,latitude_y  from centroid_cp7_sig"
query_2 = 'SELECT "CP7", "POINT_X", "POINT_Y" FROM cp7coords'
cur.execute(query_2)
SIG_POSTAL_0 = cur.fetchall()

In [24]:
import pandas as pd
SIG_POSTAL_0=pd.DataFrame(SIG_POSTAL_0,columns=['cp7','longitude(X)','latitude(Y)'])
SIG_POSTAL_0['longitude(X)']= SIG_POSTAL_0['longitude(X)'].str.replace(",",".")
SIG_POSTAL_0['latitude(Y)']= SIG_POSTAL_0['latitude(Y)'].str.replace(",",".")
 
SIG_POSTAL_0 = SIG_POSTAL_0[SIG_POSTAL_0['longitude(X)'] != '']
SIG_POSTAL_0 = SIG_POSTAL_0[SIG_POSTAL_0['latitude(Y)'] != '']

SIG_POSTAL_0


,cp7,longitude(X),latitude(Y)
0,1000-001,-9.138687,38.733097
1,1000-002,-9.139362,38.733887
2,1000-003,-9.13994,38.734596
3,1000-004,-9.138643,38.733112
4,1000-005,-9.139536,38.734165
...,...,...,...
191893,9980-039,-31.112318,39.672363
191894,9980-050,None,None
191895,9980-060,-31.11142,39.673313
191896,9980-070,-31.111685,39.672966


In [25]:
import pandas as pd
import numpy as np
df = pd.DataFrame(results_expresso_, columns=['data_criacao', 'centro', 'Giro', 'LOPTICA', 'JANELA_HORARIA', 'NOME', 'MORADA', 'CP', 'LOCALIDADE', 'COD_T_EVEN', 'DATA_EVENTO', 'LATITUDE', 'LONGITUDE','sortable_date_int'])
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]
#df=df[df['giro'].str[:3]==df['cp'].str[:3]]

# --. cálculo do centroid para cada ponto -- #


# -- considera apenas cp7 que ocorreram várias vezez--##

giro_unique_cp7 = df.groupby('cp')['data_criacao'].nunique().reset_index(name='N_Unique_cp7')

giro_unique_cp7 = giro_unique_cp7[giro_unique_cp7['N_Unique_cp7'] > 1]

giro_unique_cp7=np.array(giro_unique_cp7['cp'].unique())

df = df[df['cp'].isin(giro_unique_cp7)]

#-- excluir cp7 efetuados por várias rotas --##

unique_routes_by_cp = df.groupby('cp')['giro'].nunique().reset_index(name='Unique_Routes_Count')

unique_routes_by_cp = unique_routes_by_cp.sort_values(by='Unique_Routes_Count', ascending=False)

unique_routes_by_cp_filtered = unique_routes_by_cp[unique_routes_by_cp['Unique_Routes_Count'] ==1]

lista_excl_cp7=np.array(unique_routes_by_cp_filtered['cp'].unique())

df = df[df['cp'].isin(lista_excl_cp7)]

cp7_unique_counts = df['cp'].value_counts()

# -- cp7 para excluir quando em análise do cp4 em questão##

excl= ['2830-410','2830-411','2830-424','2830-429','2830-430','2830-432','2830-434','2830-435','2830-437',
'2830-438','2830-439','2830-441','2830-443','2830-444','2830-474','2830-476','2830-477','2830-478','2830-481','2830-581','2830-583',
'2830-592','2830-480','2830-513','2830-515','2830-519','2830-520','2830-440','2830-437','2830-066']

df = df[~df['cp'].isin(excl)]

#--- validação do giro --##

codigo_prestador='2825'
codigo_prestador_2=''
giros_excluir=['2825O','1000B']

giro_unique_counts = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_Unique_Data_Criacao')


giro_unique_counts = giro_unique_counts[
    ((giro_unique_counts['giro'].str[:4] == codigo_prestador) |
     (giro_unique_counts['giro'].str[:4] == codigo_prestador_2)) &
    (~giro_unique_counts['giro'].apply(lambda x: any(x.startswith(prefix) for prefix in giros_excluir)))
]



giro_unique_counts_average = giro_unique_counts['N_Unique_Data_Criacao'].mean()
giro_unique_counts_average
#giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > giro_unique_counts_average]
giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > 0]
giro_unique_counts

print(giro_unique_counts)

lista_val_giro=np.array(giro_unique_counts['giro'].unique())

print(lista_val_giro)

df = df[df['giro'].isin(lista_val_giro)]

#--#


# -- validação do cp4 -- 
unique_cp4_values = df['cp'].str[:4].unique()

cp_counts_by_unique_cp4 = {}
for cp4_value in unique_cp4_values:
    cp_counts_by_unique_cp4[cp4_value] = df[df['cp'].str[:4] == cp4_value]['cp'].count()

cp_counts = pd.DataFrame(list(cp_counts_by_unique_cp4.items()), columns=['CP4_Value', 'Count'])

cp_counts = cp_counts[cp_counts['Count'] > 200]  


cp_validation = np.array(cp_counts['CP4_Value'])

def expand_values(cp_values):
    expanded_values = []
    
    for value in cp_values:
        value = int(value)
        expanded_values.extend([str(value + i) for i in range(5)])
    
    return expanded_values

expanded_validation = expand_values(cp_validation)


print(cp_validation)
print(expanded_validation)

print(len(cp7_unique_counts))

     giro  N_Unique_Data_Criacao
10  2825A                      8
11  2825B                      5
12  2825C                      8
13  2825D                      8
14  2825E                      8
15  2825G                      8
16  2825H                      8
17  2825I                      8
18  2825J                      8
19  2825K                      6
20  2825N                      8
['2825A' '2825B' '2825C' '2825D' '2825E' '2825G' '2825H' '2825I' '2825J'
 '2825K' '2825N']
['2825' '2820']
['2825', '2826', '2827', '2828', '2829', '2820', '2821', '2822', '2823', '2824']
3731


In [26]:
excl= ['2830-410','2830-411','2830-424','2830-429','2830-430','2830-432','2830-434','2830-435','2830-437',
'2830-438','2830-439','2830-441','2830-443','2830-444','2830-474','2830-476','2830-477','2830-478','2830-481','2830-581','2830-583',
'2830-592','2830-480','2830-513','2830-515','2830-519','2830-520']



In [27]:
SIG_POSTAL = SIG_POSTAL_0['cp7'].str.replace(' ', '')
SIG_POSTAL = SIG_POSTAL_0[SIG_POSTAL_0['cp7'].str[:4].isin(expanded_validation)]
SIG_POSTAL = SIG_POSTAL[~SIG_POSTAL['cp7'].str[:8].isin(excl)]
SIG_POSTAL = SIG_POSTAL[~(SIG_POSTAL['longitude(X)'].isna() | (SIG_POSTAL['longitude(X)'] == ''))]
#SIG_POSTAL = SIG_POSTAL[SIG_POSTAL['cp7']=='2925-708']


SIG_POSTAL


,cp7,longitude(X),latitude(Y)
53400,2820-001,-9.195438,38.620388
53401,2820-002,-9.198908,38.625553
53402,2820-003,-9.198855,38.615345
53403,2820-004,-9.19974,38.61774
53404,2820-005,-9.200924,38.617275
...,...,...,...
54562,2829-512,-9.196728,38.66636
54563,2829-513,-9.18623,38.669426
54564,2829-514,-9.203994,38.64572
54565,2829-515,-9.198748,38.66092


In [28]:
unique_cp_values_sig = SIG_POSTAL['cp7'].nunique()

# Display the unique values in the 'cp' column
print(unique_cp_values_sig)

1167


In [29]:
from numpy import double

df_merge = pd.merge(df, SIG_POSTAL, left_on='cp', right_on='cp7', how='inner')

df_merge['latitude'] = pd.to_numeric(df_merge['latitude'], errors='coerce', downcast='float')
df_merge['longitude'] = pd.to_numeric(df_merge['longitude'], errors='coerce', downcast='float')
df_merge['latitude(Y)'] = pd.to_numeric(df_merge['latitude(Y)'], errors='coerce', downcast='float')
df_merge['longitude(X)'] = pd.to_numeric(df_merge['longitude(X)'], errors='coerce', downcast='float')

# Drop rows with NaN values in any of the specified columns
df_merge = df_merge.dropna(subset=['latitude', 'longitude', 'latitude(Y)', 'longitude(X)'])

# df_merge['latitude'] = df_merge['latitude'].astype(float)
# df_merge['longitude'] = df_merge['longitude'].astype(float)
# df_merge['latitude(Y)'] = df_merge['latitude(Y)'].astype(float)
# df_merge['longitude(X)'] = df_merge['longitude(X)'].astype(float)

print(len(df_merge))
df_merge.head()


2032


,data_criacao,centro,giro,loptica,janela_horaria,nome,morada,cp,localidade,cod_t_even,data_evento,latitude,longitude,sortable_date_int,cp7,longitude(X),latitude(Y)
1,03/01/2024,8819009,2825J,DW629671423PT,09-19,FILIPE FIALHO SILVA,AV DA LIBERDADE N 31 2 DTO,2825-862,TRAFARIA,I,03/01/2024 13:30:44,38.671463,-9.231729,20240103,2825-862,-9.231927,38.671928
4,09/01/2024,8819009,2825J,DW650453138PT,09-19,SERGIO MIGUEL MARTINS,AVENIDA DA LIBERDADE N45 R/C ESQ,2825-862,TRAFARIA,I,09/01/2024 13:44:17,38.671097,-9.231624,20240109,2825-862,-9.231927,38.671928
6,10/01/2024,8819009,2825J,FD416914934PT,None,ANABELA SIMOES,AV DA LIBERDADE N 47 2 DTO,2825-862,TRAFARIA,I,10/01/2024 14:16:19,38.670647,-9.231637,20240110,2825-862,-9.231927,38.671928
9,02/01/2024,8819009,2825N,DW629184369PT,09-19,ANDRÉ MANUEL VENTURA SOARES TE,"LARGO SAMWELL DINIS, N 4, 2 ESQ EM ALTERNATIVA...",2820-491,CHARNECA DA CAPARICA,I,02/01/2024 17:23:37,38.625957,-9.204785,20240102,2820-491,-9.204329,38.625645
10,02/01/2024,8819009,2825N,DT233376421PT,09-19,VITÓRIA,"RUA SAMWELL DINIS N 14, 1 DIREITO",2820-491,CHARNECA DA CAPARICA,I,02/01/2024 17:16:45,38.625374,-9.205357,20240102,2820-491,-9.204329,38.625645


In [30]:
unique_cp_values = df_merge['cp'].nunique()

# Display the unique values in the 'cp' column
print(unique_cp_values)

342


In [31]:
def calculate_distance(lon1, lat1, lon2, lat2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance


In [32]:
#df_merge['distance']=df_merge.apply(calculate_distance,axis=1)
df_merge['distance'] = calculate_distance(df_merge['latitude'], df_merge['longitude'], df_merge['latitude(Y)'], df_merge['longitude(X)'])
df_merge.head()

,data_criacao,centro,giro,loptica,janela_horaria,nome,morada,cp,localidade,cod_t_even,data_evento,latitude,longitude,sortable_date_int,cp7,longitude(X),latitude(Y),distance
1,03/01/2024,8819009,2825J,DW629671423PT,09-19,FILIPE FIALHO SILVA,AV DA LIBERDADE N 31 2 DTO,2825-862,TRAFARIA,I,03/01/2024 13:30:44,38.671463,-9.231729,20240103,2825-862,-9.231927,38.671928,0.055568
4,09/01/2024,8819009,2825J,DW650453138PT,09-19,SERGIO MIGUEL MARTINS,AVENIDA DA LIBERDADE N45 R/C ESQ,2825-862,TRAFARIA,I,09/01/2024 13:44:17,38.671097,-9.231624,20240109,2825-862,-9.231927,38.671928,0.097117
6,10/01/2024,8819009,2825J,FD416914934PT,None,ANABELA SIMOES,AV DA LIBERDADE N 47 2 DTO,2825-862,TRAFARIA,I,10/01/2024 14:16:19,38.670647,-9.231637,20240110,2825-862,-9.231927,38.671928,0.144217
9,02/01/2024,8819009,2825N,DW629184369PT,09-19,ANDRÉ MANUEL VENTURA SOARES TE,"LARGO SAMWELL DINIS, N 4, 2 ESQ EM ALTERNATIVA...",2820-491,CHARNECA DA CAPARICA,I,02/01/2024 17:23:37,38.625957,-9.204785,20240102,2820-491,-9.204329,38.625645,0.061182
10,02/01/2024,8819009,2825N,DT233376421PT,09-19,VITÓRIA,"RUA SAMWELL DINIS N 14, 1 DIREITO",2820-491,CHARNECA DA CAPARICA,I,02/01/2024 17:16:45,38.625374,-9.205357,20240102,2820-491,-9.204329,38.625645,0.118170


In [33]:
df=df_merge[df_merge.distance <500]
len(df)

2032

In [34]:
import pandas as pd
import numpy as np

#df.loc[:, 'N_Objetos'] = df.groupby(['giro', 'data_criacao'])['loptica'].transform('count')
df['N_Objetos'] = df.groupby(['giro', 'data_criacao'])['loptica'].transform('size')
df.loc[:, 'data_evento'] = pd.to_datetime(df['data_evento'], dayfirst=True)
df = df.sort_values(by=['giro', 'data_criacao', 'data_evento'], ascending=True)

df['cluster'] = pd.qcut(df.groupby(['giro', 'data_criacao']).cumcount(), 3, labels=[1, 2, 3])

df_aggregated = df.groupby(['cp']).agg({
    'giro': lambda x: x.value_counts().idxmax(),
    'cluster': lambda x: x.value_counts().idxmax(),
    'loptica': 'count',
    'longitude(X)': 'first',
    'latitude(Y)': 'first'
}).rename(columns={'loptica': 'count_loptica'}).reset_index()

# df_aggregated = df_aggregated[(df_aggregated['giro'] == '2835A')]

df_aggregated



,cp,giro,cluster,count_loptica,longitude(X),latitude(Y)
0,2820-002,2825G,3,10,-9.198908,38.625553
1,2820-003,2825G,2,2,-9.198855,38.615345
2,2820-006,2825G,3,3,-9.198886,38.618656
3,2820-007,2825G,3,20,-9.203456,38.616219
4,2820-008,2825G,3,5,-9.199472,38.615372
...,...,...,...,...,...,...
337,2825-886,2825J,1,3,-9.233320,38.671993
338,2825-894,2825J,1,7,-9.237618,38.670120
339,2829-506,2825D,3,5,-9.233455,38.637943
340,2829-511,2825C,1,18,-9.203247,38.667046


In [35]:

df_aggregated.loc[:,'count_loptica'] = df_aggregated['count_loptica'].astype(int)

df_aggregated.loc[:, 'giro'] = df_aggregated['giro'].astype(str)
df_aggregated.loc[:, 'cluster'] = df_aggregated['cluster'].astype(str)

df_aggregated['rota_zona'] = df_aggregated['giro'].str.cat(df_aggregated['cluster'])

df = df_aggregated.rename(columns={'cp': 'cp7', 'giro': 'rota', 'cluster': 'zona'})

df['cp4'] = df['cp7'].str[:4]

df.reset_index(drop=True, inplace=True)


df


,cp7,rota,zona,count_loptica,longitude(X),latitude(Y),rota_zona,cp4
0,2820-002,2825G,3,10,-9.198908,38.625553,2825G3,2820
1,2820-003,2825G,2,2,-9.198855,38.615345,2825G2,2820
2,2820-006,2825G,3,3,-9.198886,38.618656,2825G3,2820
3,2820-007,2825G,3,20,-9.203456,38.616219,2825G3,2820
4,2820-008,2825G,3,5,-9.199472,38.615372,2825G3,2820
...,...,...,...,...,...,...,...,...
337,2825-886,2825J,1,3,-9.233320,38.671993,2825J1,2825
338,2825-894,2825J,1,7,-9.237618,38.670120,2825J1,2825
339,2829-506,2825D,3,5,-9.233455,38.637943,2825D3,2829
340,2829-511,2825C,1,18,-9.203247,38.667046,2825C1,2829


In [36]:
from numpy import double

df_merge = pd.merge(df, SIG_POSTAL, left_on='cp7', right_on='cp7', how='right')
df_merge.drop(['longitude(X)_x', 'latitude(Y)_x'], axis=1, inplace=True)
df_merge = df_merge.rename(columns={'longitude(X)_y': 'longitude(X)', 'latitude(Y)_y': 'latitude(Y)'})
df_merge['latitude(Y)'] = df_merge['latitude(Y)'].astype(float)
df_merge['longitude(X)'] = df_merge['longitude(X)'].astype(float)
df=df_merge
df

,cp7,rota,zona,count_loptica,rota_zona,cp4,longitude(X),latitude(Y)
0,2820-001,NaN,NaN,NaN,NaN,NaN,-9.195438,38.620388
1,2820-002,2825G,3,10.0,2825G3,2820,-9.198908,38.625553
2,2820-003,2825G,2,2.0,2825G2,2820,-9.198855,38.615345
3,2820-004,NaN,NaN,NaN,NaN,NaN,-9.199740,38.617740
4,2820-005,NaN,NaN,NaN,NaN,NaN,-9.200924,38.617275
...,...,...,...,...,...,...,...,...
1162,2829-512,NaN,NaN,NaN,NaN,NaN,-9.196728,38.666360
1163,2829-513,NaN,NaN,NaN,NaN,NaN,-9.186230,38.669426
1164,2829-514,NaN,NaN,NaN,NaN,NaN,-9.203994,38.645720
1165,2829-515,NaN,NaN,NaN,NaN,NaN,-9.198748,38.660920


In [37]:
import os
from networkx import dfs_postorder_nodes
import pandas as pd
from scipy.spatial.distance import cdist
import pyproj
import numpy as np
from sklearn.cluster import KMeans


# create a transformer object to convert the coordinates to WGS84
transformer = pyproj.Transformer.from_crs(3763, 4326, always_xy=True)

# convert the POINT_X and POINT_Y columns to decimal degrees
df['longitude'], df['latitude'] = transformer.transform(df['longitude(X)'].tolist(), df['latitude(Y)'].tolist())

# get the X and Y coordinates and CP7 as arrays
coords = df[["longitude", "latitude"]].values
cp7 = df["cp7"].values.reshape(-1, 1)

# calculate the distance matrix using cdist
dist_matrix = cdist(coords, coords, "euclidean")

# set the diagonal to a high number to ignore the distance between a point and itself
dist_matrix[range(len(df)), range(len(df))] = float("inf")

# get the index of the minimum distance for each point
min_indexes = dist_matrix.argmin(axis=1)

# get the CP7 value corresponding to the minimum distance for each point
min_cp7 = cp7[min_indexes].squeeze()

# add the minimum distance and CP7 columns to the dataframe
df["min_distance"] = dist_matrix.min(axis=1)
df["min_cp7"] = min_cp7

#create a mask for blank rows in Giro, Zona, and Giro_Zona columns
mask = (df['rota'].isna()) & (df['zona'].isna()) & (df['rota_zona'].isna())

# fill the blank rows with the closest non-blank rows based on the CP7 values and their distances
for i in df[mask].index:
    min_distance = df.at[i, 'min_distance']
    closest_row_index = df[~mask].iloc[dist_matrix[i][~mask].argmin()].name
    for column in ['rota', 'zona', 'rota_zona']:
        df.at[i, column] = df.at[closest_row_index, column]
    df.at[i, 'min_distance'] = min_distance
    df.at[i, 'min_cp7'] = df.at[closest_row_index, 'cp7']


#dir_path = r"C:\Users\1420844\OneDrive - CTT - Correios de Portugal\04.Dataset_CTT\01.CP7_Rota_Zona"
#os.chdir(dir_path)


df_int=df.drop(['longitude', 'latitude', 'min_distance', 'min_cp7'],axis=1)
df_int.drop_duplicates(subset=['cp7'], keep='first', inplace=True)

#df_int.to_csv("Palmela_Completo.csv", index=False)

import pandas as pd
from sklearn.cluster import KMeans


columns = ['cp4', 'cp7', 'rota', 'zona', 'rota_zona', 'longitude(X)', 'latitude(Y)', 'longitude', 'latitude', 'min_distance', 'min_cp7']

df_final = pd.DataFrame(columns=columns)

#print(df_final)

for route in df['rota'].unique():
    # Filter the data frame to only include rows with the current Rota value
    df_route = df[df['rota'] == route]
    
    # Group the data frame by Rota_Zona and compute the centroid of each group
    #centroids = df_route.groupby(['Rota','Rota_Zona']).mean()[['longitude(X)', 'latitude(Y)']].reset_index()
    centroids = df_route.groupby(['rota', 'rota_zona']).mean(numeric_only=True)[['longitude(X)', 'latitude(Y)']].reset_index()
    centroids.columns = ['rota','rota_zona', 'centroid_X', 'centroid_Y']

    # Add a new column that specifies the centroid associated with each row
    df_route = pd.merge(df_route, centroids, on='rota_zona')

    # Use k-means to cluster the rows based on the longitude(X) and latitude(Y) columns
    kmeans = KMeans(n_clusters=len(centroids), init=centroids[['centroid_X', 'centroid_Y']], n_init=1)
    df_route['cluster'] = kmeans.fit_predict(df_route[['longitude(X)', 'latitude(Y)']]) % 3 + 1
    
    df_route = df_route.rename(columns={'rota_x': 'rota'})
    df_route = df_route.drop(['zona', 'rota_zona'], axis=1)
    df_route['zona'] = df_route['cluster'].astype(str)
    df_route['rota_zona'] = df_route['rota'].astype(str) + df_route['cluster'].astype(str)
    df_route = df_route[['cp4', 'cp7', 'rota', 'zona', 'rota_zona', 'longitude(X)', 'latitude(Y)', 'longitude', 'latitude', 'min_distance', 'min_cp7']]
    
    df_final = pd.concat([df_final, df_route], ignore_index=True)

    #df_final = df_final.append(df_route, ignore_index=True)
    

df_final=df_final.drop(['longitude', 'latitude', 'min_distance', 'min_cp7'],axis=1)
df_final.drop_duplicates(subset=['cp7'], keep='first', inplace=True)



df_final






/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_11915/2219260392.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, df_route], ignore_index=True)


,cp4,cp7,rota,zona,rota_zona,longitude(X),latitude(Y)
0,NaN,2820-001,2825G,2,2825G2,-9.195438,38.620388
1,2820,2820-002,2825G,3,2825G3,-9.198908,38.625553
2,NaN,2820-004,2825G,2,2825G2,-9.199740,38.617740
3,NaN,2820-005,2825G,2,2825G2,-9.200924,38.617275
4,2820,2820-006,2825G,2,2825G2,-9.198886,38.618656
...,...,...,...,...,...,...,...
1162,NaN,2825-481,2825B,1,2825B1,-9.235317,38.653522
1163,NaN,2825-482,2825B,1,2825B1,-9.236179,38.655220
1164,NaN,2825-483,2825B,1,2825B1,-9.235274,38.653534
1165,NaN,2825-484,2825B,1,2825B1,-9.233932,38.651020


In [38]:
import folium
import re
import random  # for generating random colors

# Create a folium map centered at the mean latitude and longitude
m = folium.Map(location=[df_final['latitude(Y)'].mean(), df_final['longitude(X)'].mean()], zoom_start=12, tiles='OpenStreetMap')

# Create a color mapping dictionary
color_mapping = {}
unique_rotas = df_final['rota'].unique()

fixed_colors = [
    "#3498DB", "#E74C3C", "#27AE60", "#F39C12", "#8E44AD",
    "#34495E", "#229954", "#7D3C98", "#8E44AD", "#D35400",
    "#E74C3C", "#F39C12", "#2E86C1", "#A569BD", "#3498DB",
    "#27AE60", "#1ABC9C", "#E67E22", "#BDC3C7", "#34495E"
]

# Iterate through unique_rotas and assign fixed colors
for index, rota in enumerate(unique_rotas):
    color_mapping[rota] = fixed_colors[index % len(fixed_colors)]

# for rota in unique_rotas:
#     color_mapping[rota] = "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Create a feature group for each rota
rota_feature_groups = {}
for rota in unique_rotas:
    rota_feature_groups[rota] = folium.FeatureGroup(name=rota)  # Create a feature group for each rota

# Iterate through the rows of df_final
for _, row in df_final.iterrows():
    rota_color = color_mapping.get(row['rota'], '#1787FE')  # Default color if not found in mapping
    folium.RegularPolygonMarker(  # Use RegularPolygonMarker to change marker shape
        location=[row['latitude(Y)'], row['longitude(X)']],
        number_of_sides=10,  # Change this value to the number of sides you want (e.g., 3 for a triangle)
        radius=3,
        popup=re.sub(r'[^a-zA-Z]+', '', str(row['cp7'])),
        color=rota_color,
        fill=True,
        fill_color=rota_color
    ).add_to(rota_feature_groups[row['rota']])  # Add to the appropriate feature group

# Add each feature group to the map
for rota, feature_group in rota_feature_groups.items():
    m.add_child(feature_group)

# Add layer control to toggle feature groups
folium.LayerControl().add_to(m)

# Display the map
m


In [39]:
# one_drive_folder = r'C:\Users\1420844\OneDrive - CTT - Correios de Portugal\04.Dataset_CTT\01.CP7_Rota_Zona'

# if os.name == 'nt':  # Windows
#     one_drive_folder = r'C:\Users\1420844\OneDrive - CTT - Correios de Portugal\04.Dataset_CTT\01.CP7_Rota_Zona'
# else:  # Assuming Unix-like (Linux, macOS)
#     one_drive_folder = "/Users/rubenrocha/Library/CloudStorage/OneDrive-CTT-CorreiosdePortugal/04.Dataset_CTT/01.CP7_Rota_Zona"
#         # Construct the full file path

one_drive_folder = os.path.expanduser('~/Downloads')

# Combine the paths using os.path.join
desired_directory = os.path.join(one_drive_folder)

# Create the directory if it doesn't exist
os.makedirs(desired_directory, exist_ok=True)

# Define the file name
file_name = str(codigo_prestador)+".csv"

# Create the full file path
file_path = os.path.join(desired_directory, file_name)

# Save the DataFrame to the CSV file
df_final.to_csv(file_path, index=False)

print(f"CSV file saved to: {file_path}")

CSV file saved to: /Users/rubenfilipemartinsdarocha/Downloads/2825.csv
